In [1]:
!pip install pandahouse
!pip install swifter

  Preparing metadata (setup.py) ... done
  Created wheel for pandahouse: filename=pandahouse-0.2.7-py2.py3-none-any.whl size=5904 sha256=945fbd377e14c07777fadd2c9ee6232bffc3aa02e3ca1d2cc501a2a96867895e
  Stored in directory: /root/.cache/pip/wheels/04/81/63/6896730711da10812121737bb505c6a8993800e99d39432522
Successfully built pandahouse
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16507 sha256=4e82acfaa503a3eb694c010bfd3b9b9747746d3b1799b257e95fa73a497a5807
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter


In [115]:
import pandas as pd
import numpy as np
import pandahouse
import swifter
import seaborn as sns
from tqdm import tqdm

# Сравнить retention по источникам траффика
Математически самым простым вариантом видится расчёт retention с помощью churn:

$retention = 1-churn\_rate$

$churn\_rate = \frac{Ушедшие \; пользователи}{Все \; пользователи}$

При этом возникает вопрос кого считать ушедшим пользоватлем

### Определение приемлемого порога ($n$ дней) для записи пользователя в отток

In [25]:
# Подключение к БД
connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator_20240320'
}

Критерии оттёкшего пользователя:
- не заходил $\ge n$ дней

Критерии реально оттёкшего пользователя
- с последней активности не заходил $\ge n$ дней
- до этого не было такого, чтобы пользователь не заходил $\ge n$ дней

При этом стоит учитывать, что статистику можно собирать лишь по пользователям, которые впервые зашли $\ge n$ дней назад

In [218]:
def get_churn_stats(churn_break_n, db='simulator_20240320'):
  '''
  Функция для получения статистики по оттоку по заданному порогу, по которому считать пользователя оттёкшим

  Arguments:
    churn_break_n: int
      Порог, при котором пользователь считается оттёкшим (отток при date_break>=churn_break_n)
    db: str, default 'simulator_20240320'
      БД для подключения
  --------
  Returns:
    pd.DataFrame
      churn_break_n - порог, при котором пользователь считается оттёкшим
      uniq_users - количество уникальных пользователей, которых можно использовать для расчёта оттока
      churn - всего уникальных пользователей, которых можно было считать оттёкшими
      real_churn - уникальных пользователей, которых считаем реально оттёкшими
      churn_precision - часть реально оттёкших пользователей (от тех, кого можно было бы считать оттёкшим на каком-то этапе)
      churn_rate - по пользователям, которые считаются реально ушедшими
  '''

  q = f"""
  WITH
    t_user_activity_dates_1 AS (
      -- Дни, когда активничал пользователь
      SELECT DISTINCT
        user_id,
        DATE(time) AS date
      FROM {db}.feed_actions
    ),
    t_user_activity_dates_2 AS (
      -- Дата первой активности пользователей
      SELECT
        user_id,
        date,
        MIN(date) OVER (
          PARTITION BY user_id
        ) AS first_date
      FROM t_user_activity_dates_1
    ),
    t_user_activity_dates AS (
      -- Отбрасываем юзеров, по которым не получится собрать статистику по их оттоку
      SELECT
        user_id,
        date,
        first_date
      FROM t_user_activity_dates_2
      WHERE DATE(now()) - first_date >= {n}
    ),
    t_is_final_churn_1 AS (
      -- Последняя дата активности пользователя
      SELECT
        user_id,
        MAX(date) AS last_date
      FROM t_user_activity_dates
      GROUP BY user_id
    ),
    t_is_final_churn AS (
      -- Считать ли пользователя оттёкшим по последнему временому перерыву
      SELECT
        user_id,
        last_date,
        DATE(now()) - last_date AS date_break,
        date_break >= {n} AS is_final_churn
      FROM t_is_final_churn_1
    ),
    t_prev_dates AS (
      -- Даты предыдущей активности пользователей
      SELECT
        user_id,
        date,
        MIN(date) OVER (
          PARTITION BY user_id
          ORDER BY date
          ROWS BETWEEN 1 PRECEDING AND CURRENT ROW
        ) AS prev_date
      FROM t_user_activity_dates
    ),
    t_user_date_breaks AS (
      -- Временные перерывы пользователей (не включая последний)
      SELECT
        user_id,
        date,
        prev_date,
        date - prev_date AS date_break,
        date_break >= {n} AS is_churn
      FROM t_prev_dates
    ),
    t_final_churn_users AS (
      SELECT DISTINCT user_id
      FROM t_is_final_churn
      WHERE is_final_churn = 1
    ),
    t_not_final_churn_users AS (
      SELECT DISTINCT user_id
      FROM t_user_date_breaks
      WHERE is_churn = 1
    ),
    t_all_churn AS (
      -- Все пользователи, которых записали в отток
      SELECT DISTINCT user_id
      FROM
      (
        (
          SELECT *
          FROM t_final_churn_users
        )
        UNION ALL
        (
          SELECT *
          FROM t_not_final_churn_users
        )
      )
    ),
    t_real_churn AS (
      -- Реальный отток (пользователи, которые до этого не уходили на количество дней >= порогового)
      SELECT *
      FROM t_final_churn_users
        LEFT ANTI JOIN t_not_final_churn_users
          USING user_id
    ),
    t_churns AS (
      -- Итоговая таблица
      SELECT
        {n} AS churn_break_n,
        uniq_users,
        all_churn AS churn,
        real_churn,
        real_churn / all_churn AS churn_precision,
        real_churn / uniq_users AS churn_rate
      FROM (
        SELECT COUNT(*) AS real_churn
        FROM t_real_churn
      ) AS l
        CROSS JOIN (
            SELECT COUNT(*) AS all_churn
            FROM t_all_churn
          ) AS r1
        CROSS JOIN (
          SELECT COUNT(DISTINCT user_id) AS uniq_users
          FROM t_user_activity_dates
        ) AS r2
    )

  SELECT *
  FROM t_churns
  """
  return pandahouse.read_clickhouse(q, connection=connection)

In [219]:
# Запрос для определения удачности порога, при котором пользователь записывается в отток
n = 7  # Порог в днях для того, чтобы считать пользователя оттёкшим (отток при date_break>=n)

get_churn_stats(n)

,churn_break_n,uniq_users,churn,real_churn,churn_precision,churn_rate
0,7,98018,79195,16592,0.209508,0.169275


Перебор $n$

In [195]:
# Всего дней в БД
q = """
  SELECT DATE(now()) - DATE(MIN(time)) AS min_date
  FROM {db}.feed_actions
  """

db_days = pandahouse.read_clickhouse(q, connection=connection).values[0][0]
db_days

57

In [171]:
ns = list(range(7, db_days, 7))  # Возможные пороги для того, чтобы считать пользователя ушедшим
ns

[7, 14, 21, 28, 35, 42, 49, 56]

In [220]:
n = ns[0]
churn_th_df = get_churn_stats(n)

for n in tqdm(ns[1:-1]):
  churn_th_df = pd.concat([churn_th_df, get_churn_stats(n)], ignore_index=True)
churn_th_df

100%|██████████| 6/6 [00:32<00:00,  5.43s/it]


,churn_break_n,uniq_users,churn,real_churn,churn_precision,churn_rate
0,7,98018,79195,16592,0.209508,0.169275
1,14,84529,39162,15172,0.387416,0.179489
2,21,70067,14313,6728,0.470062,0.096022
3,28,57153,5061,2798,0.552855,0.048956
4,35,42175,1575,1026,0.651429,0.024327
5,42,28045,375,288,0.768000,0.010269
6,49,14710,50,45,0.900000,0.003059
